In [2]:
from sklearn.neighbors import kneighbors_graph
import numpy as np
import pandas as pd
import math
import datetime
import os
import shutil
import torch
import torch_geometric.transforms as T
from torch_geometric.data import Data, InMemoryDataset

In [11]:
InputFolderName = "/home/melnike/Cyto/Python_unbiased/" 
KNN_K = 100 # square root of the average number of cells (12000)

In [10]:
## Import image name list.
Region_filename = InputFolderName + "ImageNameList.txt"
region_name_list = pd.read_csv(
        Region_filename,
        sep="\t",  # tab-separated
        header=None,  # no heading row
        names=["Image"],  # set our own names for the columns
    )

In [13]:
## Below is for generation of topology structures (edges) of cellular spatial graphs.
ThisStep_OutputFolderName = "/home/melnike/Cyto/Python_unbiased/Step1_Output/"
if os.path.exists(ThisStep_OutputFolderName):
    shutil.rmtree(ThisStep_OutputFolderName)
os.makedirs(ThisStep_OutputFolderName)

print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
print("Constructing topology structures of KNN graphs...")
for graph_index in range(0, len(region_name_list)):

    print(f"This is image-{graph_index}")
    # Import target graph x/y coordinates.
    region_name = str(region_name_list.Image[graph_index])
    GraphCoord_filename = InputFolderName + region_name + "_Coordinates.txt"
    x_y_coordinates = np.loadtxt(GraphCoord_filename, dtype='float', delimiter="\t")

    K = KNN_K
    KNNgraph_sparse = kneighbors_graph(x_y_coordinates, K, mode='connectivity', include_self=False, n_jobs=-1)  #should NOT include itself as a nearest neighbor. Checked. "-1" means using all available cores.
    KNNgraph_AdjMat = KNNgraph_sparse.toarray()
    # Make the graph to undirected.
    KNNgraph_AdjMat_fix = KNNgraph_AdjMat + KNNgraph_AdjMat.T  #2min and cost one hundred memory.
    # Extract and write the edge index of the undirected graph.
    KNNgraph_EdgeIndex = np.argwhere(KNNgraph_AdjMat_fix > 0)  #1min
    filename0 = ThisStep_OutputFolderName + region_name + "_EdgeIndex.txt"
    np.savetxt(filename0, KNNgraph_EdgeIndex, delimiter='\t', fmt='%i')  #save as integers. Checked the bidirectional edges.
    
print("All topology structures have been generated!")
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

2024-02-18 06:19:40
Constructing topology structures of KNN graphs...
This is image-0
This is image-1
This is image-2
All topology structures have been generated!
2024-02-18 06:20:04


In [14]:
## Below is for generation of node attribute matrices of cellular spatial graphs.
print("Generating node attribute matrices of KNN graphs...")
cell_type_vec = []
num_nodes = []
for graph_index in range(0, len(region_name_list)):

    print("This is processing of CellTypeLabel")
    region_name = str(region_name_list.Image[graph_index])
    # Import cell type label.
    CellType_filename = InputFolderName + region_name + "_CellTypeLabel.txt"
    cell_type_label = pd.read_csv(
        CellType_filename,
        sep="\t",  # tab-separated
        header=None,  # no heading row
        names=["cell_type"],  # set our own names for the columns
    )
    cell_type_vec.extend(cell_type_label["cell_type"].values.tolist())
    num_nodes.append(len(cell_type_label))

cell_type_vec_uniq = list(set(cell_type_vec))  # generate a vector of unique cell types and store it to .txt for final illustration.
CellTypeVec_filename = ThisStep_OutputFolderName + "UniqueCellTypeList.txt"
with open(CellTypeVec_filename, 'w') as fp:
    print("This is processing of MaxNumNodes")
    for item in cell_type_vec_uniq:
        # write each item on a new line
        fp.write("%s\n" % item)

max_nodes = math.ceil(max(num_nodes))  # generate the max number of cells and store this value to .txt for the next step.
MaxNumNodes_filename = ThisStep_OutputFolderName + "MaxNumNodes.txt"
with open(MaxNumNodes_filename, 'w') as fp1:
    fp1.write("%i\n" % max_nodes)

print("All files have been generated!")
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

Generating node attribute matrices of KNN graphs...
This is processing of CellTypeLabel
This is processing of CellTypeLabel
This is processing of CellTypeLabel
This is processing of MaxNumNodes
All files have been generated!
2024-02-18 06:22:15


In [15]:
for graph_index in range(0, len(region_name_list)):

    print(f"This is image-{graph_index}")
    region_name = str(region_name_list.Image[graph_index])
    # import cell type label.
    CellType_filename = InputFolderName + region_name + "_CellTypeLabel.txt"
    cell_type_label = pd.read_csv(
        CellType_filename,
        sep="\t",  # tab-separated
        header=None,  # no heading row
        names=["cell_type"],  # set our own names for the columns
    )

    # initialize a zero-valued numpy matrix.
    node_attr_matrix = np.zeros((len(cell_type_label), len(cell_type_vec_uniq)))
    for cell_ind in range(0, len(cell_type_label)):
        # get the index of the current cell.
        type_index = cell_type_vec_uniq.index(cell_type_label["cell_type"][cell_ind])
        node_attr_matrix[cell_ind, type_index] = 1  # make the one-hot vector for each cell.

    filename1 = ThisStep_OutputFolderName + region_name + "_NodeAttr.txt"
    np.savetxt(filename1, node_attr_matrix, delimiter='\t', fmt='%i')  #save as integers.

print("All node attribute matrices have been generated!")
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

This is image-0
This is image-1
This is image-2
All node attribute matrices have been generated!
2024-02-18 06:22:24


In [16]:
## Below is for transforming input graphs into the data structure required by deep geometric learning. 
print("Start graph data structure transformation...")
# Construct ordinary Python list to hold all input graphs.
data_list = []
for i in range(0, len(region_name_list)):
    region_name = region_name_list.Image[i]

    # Import network topology.
    EdgeIndex_filename = ThisStep_OutputFolderName + str(region_name) + "_EdgeIndex.txt"
    edge_ndarray = np.loadtxt(EdgeIndex_filename, dtype = 'int64', delimiter = "\t")
    edge_index = torch.from_numpy(edge_ndarray)
    #print(edge_index.type()) #should be torch.LongTensor due to its dtype=torch.int64

    # Import node attribute.
    NodeAttr_filename = ThisStep_OutputFolderName + str(region_name) + "_NodeAttr.txt"
    x_ndarray = np.loadtxt(NodeAttr_filename, dtype='float32', delimiter="\t")  #should be float32 not float or float64.
    x = torch.from_numpy(x_ndarray)
    #print(x.type()) #should be torch.FloatTensor not torch.DoubleTensor.
    
    data = Data(x=x, edge_index=edge_index.t().contiguous())
    data_list.append(data)

print("All graphs have been generated!")
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

Start graph data structure transformation...
All graphs have been generated!
2024-02-18 06:22:28


In [17]:
import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.loader import DenseDataLoader
from torch_geometric.nn import DenseGraphConv, dense_mincut_pool
from torch_geometric.data import InMemoryDataset
import torch_geometric.transforms as T
import os
import numpy as np
import pandas as pd
import datetime
import csv
import shutil

In [18]:
Image_Name = 1
Num_TCN = 6
Num_Run = 20
Num_Epoch = 3000
Embedding_Dimension = 128
Learning_Rate = 0.0001
Loss_Cutoff = -0.6

In [23]:
Image_Name = 3
Num_TCN = 6
Num_Run = 20
Num_Epoch = 3000
Embedding_Dimension = 128
Learning_Rate = 0.0001
Loss_Cutoff = -0.2

In [24]:
Region_filename = InputFolderName + "ImageNameList.txt"
region_name_list = pd.read_csv(
        Region_filename,
        sep="\t",  # tab-separated
        header=None,  # no heading row
        names=["Image"],  # set our own names for the columns
    )

LastStep_OutputFolderName = "./Cyto/Python_unbiased/Step1_Output/"
MaxNumNodes_filename = LastStep_OutputFolderName + "MaxNumNodes.txt"
max_nodes = np.loadtxt(MaxNumNodes_filename, dtype = 'int64', delimiter = "\t").item()

class SpatialOmicsImageDataset(InMemoryDataset):
    def __init__(self, root, transform=None, pre_transform=None):
        super(SpatialOmicsImageDataset, self).__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return ['SpatialOmicsImageDataset.pt']

    def download(self):
        pass
    
    def process(self):
        # Read data_list into huge `Data` list.
        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

dataset = SpatialOmicsImageDataset(LastStep_OutputFolderName, transform=T.ToDense(max_nodes))

class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels, hidden_channels=Embedding_Dimension):
        super(Net, self).__init__()

        self.conv1 = DenseGraphConv(in_channels, hidden_channels)
        num_cluster1 = Num_TCN   #This is a hyperparameter.
        self.pool1 = Linear(hidden_channels, num_cluster1)

    def forward(self, x, adj, mask=None):

        x = F.relu(self.conv1(x, adj, mask))
        s = self.pool1(x)  #Here "s" is a non-softmax tensor.
        x, adj, mc1, o1 = dense_mincut_pool(x, adj, s, mask)
        #Save important clustering results_1.
        ClusterAssignTensor_1 = s
        ClusterAdjTensor_1 = adj

        return F.log_softmax(x, dim=-1), mc1, o1, ClusterAssignTensor_1, ClusterAdjTensor_1


def train(epoch):
    model.train()
    loss_all = 0

    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out, mc_loss, o_loss, _, _ = model(data.x, data.adj, data.mask)
        loss = mc_loss + o_loss
        loss.backward()
        loss_all += loss.item()
        optimizer.step()
    return loss_all

# Extract a single graph for TCN learning.
ThisStep_OutputFolderName = "./Step2_Output_" + str(Image_Name) + "/"
os.makedirs(ThisStep_OutputFolderName, exist_ok=True)

train_index = [region_name_list["Image"].values.tolist().index(Image_Name)]
train_dataset = dataset[train_index]
train_loader = DenseDataLoader(train_dataset, batch_size=1)
all_sample_loader = DenseDataLoader(train_dataset, batch_size=1)
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

2024-02-18 13:36:21


In [25]:
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
run_number = 1
while run_number <= Num_Run:  #Generate multiple independent runs for ensemble.

    print(f"This is Run{run_number:02d}")

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = Net(dataset.num_features, 1).to(device)  #Initializing the model.
    optimizer = torch.optim.Adam(model.parameters(), lr=Learning_Rate)
    
    RunFolderName = ThisStep_OutputFolderName + "Run" + str(run_number)
    if os.path.exists(RunFolderName):
        shutil.rmtree(RunFolderName)
    os.makedirs(RunFolderName)  #Creating the Run folder.
    
    filename_0 = RunFolderName + "/Epoch_UnsupervisedLoss.csv"
    headers_0 = ["Epoch", "UnsupervisedLoss"]
    with open(filename_0, "w", newline='') as f0:
        f0_csv = csv.writer(f0)
        f0_csv.writerow(headers_0)

    previous_loss = float("inf")  #Initialization.
    for epoch in range(1, Num_Epoch+1):  #Specify the number of epoch in each independent run.
        train_loss = train(epoch)
        #print(f"Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}")
        with open(filename_0, "a", newline='') as f0:
            f0_csv = csv.writer(f0)
            f0_csv.writerow([epoch, train_loss])
        
        if train_loss == 0 and train_loss == previous_loss:  #If two consecutive losses are both zeros, the learning gets stuck.
            break  #stop the training.
        else:
            previous_loss = train_loss

    print(f"Final train loss is {train_loss:.4f}")
    if train_loss >= Loss_Cutoff:   #This is an empirical cutoff of the final loss to avoid underfitting.
        shutil.rmtree(RunFolderName)  #Remove the specific folder and all files inside it for re-creating the Run folder.
        continue  #restart this run.

    #Extract the soft TCN assignment matrix using the trained model.
    for EachData in all_sample_loader:
        EachData = EachData.to(device)
        TestModelResult = model(EachData.x, EachData.adj, EachData.mask)

        ClusterAssignMatrix1 = TestModelResult[3][0, :, :]
        ClusterAssignMatrix1 = torch.softmax(ClusterAssignMatrix1, dim=-1)  #Checked, consistent with the built-in function "dense_mincut_pool".
        ClusterAssignMatrix1 = ClusterAssignMatrix1.detach().numpy()
        filename1 = RunFolderName + "/TCN_AssignMatrix1.csv"
        np.savetxt(filename1, ClusterAssignMatrix1, delimiter=',')

        ClusterAdjMatrix1 = TestModelResult[4][0, :, :]
        ClusterAdjMatrix1 = ClusterAdjMatrix1.detach().numpy()
        filename2 = RunFolderName + "/TCN_AdjMatrix1.csv"
        np.savetxt(filename2, ClusterAdjMatrix1, delimiter=',')

        NodeMask = EachData.mask
        NodeMask = np.array(NodeMask)
        filename3 = RunFolderName + "/NodeMask.csv"
        np.savetxt(filename3, NodeMask.T, delimiter=',', fmt='%i')  #save as integers.

    run_number = run_number + 1

print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

2024-02-18 13:36:30
This is Run01
Final train loss is -0.6614
This is Run02
Final train loss is -0.3562
This is Run03
Final train loss is 0.0879
This is Run03
Final train loss is -0.3768
This is Run04
Final train loss is -0.3649
This is Run05
Final train loss is -0.2315
This is Run06
Final train loss is -0.2358
This is Run07
Final train loss is -0.2252
This is Run08
Final train loss is -0.6366
This is Run09
Final train loss is -0.3335
This is Run10
Final train loss is -0.6497
This is Run11
Final train loss is -0.2212
This is Run12
Final train loss is -0.6746
This is Run13
Final train loss is -0.6618
This is Run14
Final train loss is -0.2098
This is Run15
Final train loss is -0.3615
This is Run16
Final train loss is -0.3579
This is Run17
Final train loss is -0.6539
This is Run18
Final train loss is -0.3654
This is Run19
Final train loss is 0.0879
This is Run19
Final train loss is 0.0879
This is Run19
Final train loss is -0.6423
This is Run20
Final train loss is -0.6328
2024-02-18 17:41:

In [26]:
# step 3 is executed in R, return here after
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sci_palettes
import os
import shutil
import matplotlib as mpl

In [27]:
mpl.rcParams['pdf.fonttype'] = 42     # make text in plot editable in AI.
#print(sci_palettes.PALETTES.keys()) # used for checking all color schemes of different journals.
print('------')
#sci_palettes.PALETTES["simpsons_springfield"]     # see detailed color code
sci_palettes.register_cmap("simpsons_springfield")     # register a specific palette for TCN coloring.

------


In [35]:
## Hyperparameters
InputFolderName = "/home/melnike/Cyto/Python_unbiased/"
Image_Name = '3'


## Import the cell type list used for matching color palettes across different cell type plots.
unique_cell_type_df = pd.read_csv(
        "./Cyto/Python_unbiased/Step1_Output/UniqueCellTypeList.txt",
        sep="\t",  # tab-separated
        header=None,  # no heading row
        names=["UniqueCellType"],  # set our own names for the columns
    )
UniqueCellType_vec = unique_cell_type_df['UniqueCellType'].values.tolist()
print('Cell type is imported')

## Import target cellular spatial graph x/y coordinates.
GraphCoord_filename = InputFolderName + Image_Name + "_Coordinates.txt"
x_y_coordinates = pd.read_csv(
        GraphCoord_filename,
        sep="\t",  # tab-separated
        header=None,  # no heading row
        names=["x_coordinate", "y_coordinate"],  # set our own names for the columns
    )
print('Cellular spatial graph is imported')
target_graph_map = x_y_coordinates
#target_graph_map["y_coordinate"] = 0 - target_graph_map["y_coordinate"]  # for consistent with original paper. Don't do this is also ok.


## Import cell type label.
CellType_filename = InputFolderName + Image_Name + "_CellTypeLabel.txt"
cell_type_label = pd.read_csv(
        CellType_filename,
        sep="\t",  # tab-separated
        header=None,  # no heading row
        names=["cell_type"],  # set our own names for the columns
    )
print('Cell type label is imported')

# Add cell type labels to target graph x/y coordinates.
target_graph_map["Cell_Type"] = cell_type_label.cell_type
# Below is for matching color palettes across different cell type plots.
target_graph_map["Cell_Type"] = pd.Categorical(target_graph_map["Cell_Type"], UniqueCellType_vec)


## Import the final TCN labels to target graph x/y coordinates.
LastStep_OutputFolderName = "./Cyto/Step3_Output_" + Image_Name + "/"
target_graph_map["TCN_Label"] = np.loadtxt(LastStep_OutputFolderName + "TCNLabel_MajorityVoting.csv", dtype='int', delimiter=",")
# Converting integer list to string list for making color scheme discrete.
target_graph_map.TCN_Label = target_graph_map.TCN_Label.astype(str)

print('Final TCN is imported')

Cell type is imported
Cellular spatial graph is imported
Cell type label is imported
Final TCN is imported


In [36]:
#-----------------------------------------Generate plots-------------------------------------------------#
ThisStep_OutputFolderName = "./Step4_Output_" + Image_Name + "/"
if os.path.exists(ThisStep_OutputFolderName):
    shutil.rmtree(ThisStep_OutputFolderName)
os.makedirs(ThisStep_OutputFolderName)

print('Plot TCN label')
## Plot x/y map with "TCN_Label" coloring.
TCN_plot = sns.scatterplot(x="x_coordinate", y="y_coordinate", data=target_graph_map, hue="TCN_Label", palette="simpsons_springfield", alpha=1.0, s=20.0, legend="full")   # 20 colors at maximum.
# Hide all four spines
TCN_plot.spines.right.set_visible(False)
TCN_plot.spines.left.set_visible(False)
TCN_plot.spines.top.set_visible(False)
TCN_plot.spines.bottom.set_visible(False)
TCN_plot.set(xticklabels=[])  # remove the tick label.
TCN_plot.set(yticklabels=[])
TCN_plot.set(xlabel=None)  # remove the axis label.
TCN_plot.set(ylabel=None)
TCN_plot.tick_params(bottom=False, left=False)  # remove the ticks.
# Place legend outside top right corner of the CURRENT plot
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', borderaxespad=0)
# Save the CURRENT figure.
TCN_fig_filename1 = ThisStep_OutputFolderName + "TCN_" + Image_Name + ".pdf"
plt.savefig(TCN_fig_filename1)
TCN_fig_filename2 = ThisStep_OutputFolderName + "TCN_" + Image_Name + ".png"
plt.savefig(TCN_fig_filename2)
plt.close()
print('TCN plots are done')

Plot TCN label
TCN plots are done


In [37]:
print('plot spatial coordinates')
## Plot x/y map with "Cell_Type" coloring.
num_unique_categories = len(target_graph_map["Cell_Type"].unique())
palette = sns.color_palette("husl", 8)
CellType_plot = sns.scatterplot(x="x_coordinate", y="y_coordinate", data=target_graph_map, hue="Cell_Type", palette=palette, alpha=1.0, s=20.0, legend="full")  # 30 colors at maximum.
# Hide all four spines
CellType_plot.spines.right.set_visible(False)
CellType_plot.spines.left.set_visible(False)
CellType_plot.spines.top.set_visible(False)
CellType_plot.spines.bottom.set_visible(False)
CellType_plot.set(xticklabels=[])  # remove the tick label.
CellType_plot.set(yticklabels=[])
CellType_plot.set(xlabel=None)  # remove the axis label.
CellType_plot.set(ylabel=None)
CellType_plot.tick_params(bottom=False, left=False)  # remove the ticks.
# Place legend outside top right corner of the CURRENT plot
plt.legend(bbox_to_anchor=(1, 1), loc='upper left', borderaxespad=0)
# Save the CURRENT figure.
CellType_fig_filename1 = ThisStep_OutputFolderName + "CellType_" + Image_Name + ".pdf"
plt.savefig(CellType_fig_filename1)
CellType_fig_filename2 = ThisStep_OutputFolderName + "CellType_" + Image_Name + ".png"
plt.savefig(CellType_fig_filename2)
plt.close()


## Export result dataframe: "target_graph_map".
TargetGraph_dataframe_filename = ThisStep_OutputFolderName + "ResultTable_" + Image_Name + ".csv"
target_graph_map.to_csv(TargetGraph_dataframe_filename, na_rep="NULL", index=False) #remove row index.

print('All done')

plot spatial coordinates
All done
